In [12]:
#ОБРУЧАЛКА
import pandas as pd
#загужаем файл остатков и отметает лишнее
ost_obr=pd.read_csv('tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
               usecols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Проект"])
ost_obr=ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ'])].sort_values('КодСклада')
del ost_obr['Проект']

#делим файл на две части:магазы и центральные склады
ost_obr1=ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr=ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr=ost_obr[['КодСклада','ТоварНо']]
ost_obr1=ost_obr1[['СерийныйНомер','ТоварНо']]

#обработка заборников верхние пустые строки в загружаемом файле должны быть удалены
zab=pd.read_excel('zab.xlsx')
zab=zab[["Серийный Но.","Товар Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.columns = ['СерийныйНомер','ТоварНо','КодСклада','ЗЛ']

#делим файл на две части:необиркованные и без СН на машине
zab_0=zab[zab.СерийныйНомер.isnull()]
zab_0=zab_0[['КодСклада','ТоварНо']]
zab=zab[zab.СерийныйНомер.str.contains('СН',na=False)]
zab=zab[['СерийныйНомер','КодСклада']]

#проставляем магаз в серийники, которые на 4001/3075/опте
df=pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1=df[['КодСклада','ТоварНо']]

#соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1,zab_0])

#догружаем размер/артикул/толщ.плетения и тд
df=pd.read_excel('карточки.xlsx',sheet_name='обручи',header=6)
result=pd.merge(result,df,left_on=['ТоварНо'],right_on=['Номер'],how='left')
del result['Номер']
result.set_index('КодСклада',drop=True, inplace=True)

#сохраняем в эксель. нужно доработать сохранение в уже существующий эксель
with pd.ExcelWriter(r'C:\Остатки\обручи.xlsx') as writer:
    result.to_excel(writer, sheet_name='остатки')

In [2]:
with pd.ExcelWriter(r'C:\Остатки\тест\болванка.xlsx',engine=None, mode='a') as writer:
    result.to_excel(writer, sheet_name='остатки')

ValueError: Append mode is not supported with xlsxwriter!

In [3]:
import xlsxwriter

workbook = xlsxwriter.Workbook(r'C:\Остатки\тест\болванка.xlsx',{'constant_memory': False})
worksheet = workbook.add_worksheet()



In [14]:
#БУКВЫ И ЗОДИАКИ ВЕЗДЕ ПОСТАВИТЬ ФИЛЬТР ПОДВЕС ДЕКОРАТИВНЫЙ
import pandas as pd
#загужаем файл остатков и отметает лишнее
ost_obr=pd.read_csv('tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
               usecols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","ТоварноеНаправление","Проект"])
ost_obr=ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ'])].sort_values('КодСклада')
del ost_obr['Проект']

#делим файл на две части:магазы и центральные склады
ost_obr1=ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr=ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr=ost_obr[['КодСклада','ТоварНо','ТоварноеНаправление']]
ost_obr1=ost_obr1[['СерийныйНомер','ТоварНо','ТоварноеНаправление']]

#обработка заборников верхние пустые строки в загружаемом файле должны быть удалены
zab=pd.read_excel('zab.xlsx')
zab=zab[["Серийный Но.","Товар Но.","Склад-приемник","Номер ЗЛ","Товарное направление"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.columns = ['СерийныйНомер','ТоварНо','КодСклада','ЗЛ','ТоварноеНаправление']

#делим файл на две части:необиркованные и без СН на машине
zab_0=zab[zab.СерийныйНомер.isnull()]
zab_0=zab_0[['КодСклада','ТоварНо','ТоварноеНаправление']]
zab=zab[zab.СерийныйНомер.str.contains('СН',na=False)]
zab=zab[['СерийныйНомер','КодСклада']]

#проставляем магаз в серийники, которые на 4001/3075/опте
df=pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1=df[['КодСклада','ТоварНо','ТоварноеНаправление']]

#соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1,zab_0])

#догружаем размер/артикул/толщ.плетения и тд
df=pd.read_excel('карточки.xlsx',sheet_name='буки',header=6)
result=pd.merge(result,df,left_on=['ТоварНо'],right_on=['Номер'],how='left')
del result['Номер']
result.set_index('КодСклада',drop=True, inplace=True)

# Очищаем описание 3 от ненужного мусора
result['Описание 3']=result['Описание 3'].apply(lambda a: a[:a.find(' ')] if a.find(' ')>0 else a )

#сохраняем в эксель. нужно доработать сохранение в уже существующий эксель
with pd.ExcelWriter(r'C:\Остатки\буквыизодиаки.xlsx') as writer:
    result.to_excel(writer)

In [7]:
#Белое
import pandas as pd
#загужаем файл остатков и отметает лишнее
ost_obr=pd.read_csv('tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
                    usecols=["КодСклада", "НазваниеСклада", "Тип1","ПоставщикАрт",
                                "РазмерИзделия","ТоварНо","СерийныйНомер","Проект"])
ost_obr=ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ'])].sort_values('КодСклада')
del ost_obr['Проект']

#делим файл на две части:магазы и центральные склады
ost_obr1=ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr=ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
del ost_obr1['КодСклада']

#обработка заборников верхние пустые строки в загружаемом файле должны быть удалены
zab=pd.read_excel('zab.xlsx')
zab=zab[["Серийный Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.columns = ['СерийныйНомер','КодСклада','ЗЛ']
del zab['ЗЛ']

#проставляем магаз в серийники, которые на 4001/3075/опте
ost_obr1=pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1=ost_obr1[["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Тип1","ПоставщикАрт","РазмерИзделия"]]
#соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1])
result.set_index('КодСклада',drop=True, inplace=True)


#сохраняем в эксель. нужно доработать сохранение в уже существующий эксель
with pd.ExcelWriter(r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\остатки.xlsx') as writer:
    result.to_excel(writer)

In [13]:
#ЛЮБОЙ ТОВАР
import pandas as pd
#загужаем файл остатков и отметает лишнее
ost_obr=pd.read_csv('tmp.csv', delimiter=';', encoding='utf-8',low_memory=False,
                    usecols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Проект"])
ost_obr=ost_obr[ost_obr.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ'])].sort_values('КодСклада')
del ost_obr['Проект']

#делим файл на две части:магазы и центральные склады
ost_obr1=ost_obr[ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr=ost_obr[~ost_obr.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
ost_obr=ost_obr[["КодСклада",  "ТоварНо","СерийныйНомер"]]
del ost_obr1['КодСклада']

#обработка заборников верхние пустые строки в загружаемом файле должны быть удалены
zab=pd.read_excel('zab.xlsx')
zab=zab[["Серийный Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.columns = ['СерийныйНомер','КодСклада','ЗЛ']
del zab['ЗЛ']

#проставляем магаз в серийники, которые на 4001/3075/опте
ost_obr1=pd.merge(ost_obr1,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
ost_obr1=ost_obr1[["КодСклада",  "ТоварНо","СерийныйНомер"]]


#соединяем остатки на магазах,на машине, то что числилось на 4001/3075/опте
result = pd.concat([ost_obr,ost_obr1])
result.set_index('КодСклада',drop=True, inplace=True)


#сохраняем в эксель. нужно доработать сохранение в уже существующий эксель
with pd.ExcelWriter(r'C:\Остатки\остатки.xlsx') as writer:
    result.to_excel(writer)

In [6]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False,
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.
  
    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]
  
    Returns: None
    """
    from openpyxl import load_workbook
  
    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')
  
    writer = pd.ExcelWriter(filename, engine='openpyxl')
  
    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)
  
        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row
  
        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)
  
        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass
  
    if startrow is None:
        startrow = 0
  
    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)
  
    # save the workbook
    writer.save()
